In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [2]:
bnf_1min = pd.read_csv("../data/bnf_min.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [3]:
bnf_1min.tail()

,datetime,open,high,low,close
727464,2024-12-31 15:25:00+05:30,50867.6500,50871.2500,50855.0500,50864.5000
727465,2024-12-31 15:26:00+05:30,50865.1500,50885.8500,50865.1500,50875.5000
727466,2024-12-31 15:27:00+05:30,50875.2500,50892.7500,50865.0000,50888.6500
727467,2024-12-31 15:28:00+05:30,50891.2500,50906.3500,50884.6000,50884.6000
727468,2024-12-31 15:29:00+05:30,50886.7500,50902.1500,50857.2000,50884.3000


In [4]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [5]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2023, 3, 1),
 datetime.date(2019, 3, 20),
 datetime.date(2021, 1, 6),
 datetime.date(2018, 12, 12),
 datetime.date(2017, 10, 13),
 datetime.date(2019, 8, 21),
 datetime.date(2017, 4, 11),
 datetime.date(2024, 8, 22),
 datetime.date(2024, 7, 25),
 datetime.date(2020, 5, 22),
 datetime.date(2024, 3, 21),
 datetime.date(2019, 5, 21),
 datetime.date(2019, 11, 8),
 datetime.date(2019, 1, 7),
 datetime.date(2019, 1, 10),
 datetime.date(2023, 8, 23),
 datetime.date(2019, 7, 23),
 datetime.date(2024, 5, 17),
 datetime.date(2022, 6, 15),
 datetime.date(2021, 1, 15),
 datetime.date(2018, 12, 6),
 datetime.date(2021, 5, 21),
 datetime.date(2024, 9, 27),
 datetime.date(2018, 7, 24),
 datetime.date(2023, 4, 27),
 datetime.date(2020, 6, 5),
 datetime.date(2020, 9, 29),
 datetime.date(2017, 12, 4),
 datetime.date(2021, 1, 8),
 datetime.date(2022, 5, 12),
 datetime.date(2021, 8, 16),
 datetime.date(2023, 7, 17),
 datetime.date(2020, 12, 11),
 datetime.date(2021, 9, 14),
 datetime.date(2

In [6]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close"}
    df = df.rename(column_mapping)

    return df

In [78]:
import pandas as pd
import numpy as np

# Calculate the RSI using vectorized operations
def calculate_rsi(df, period=14):
    delta = df['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=period, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to detect RSI divergence without loops
def calculate_signals(df, look_back=10):
    df['RSI'] = calculate_rsi(df)
    
    # Initialize the signal columns
    df['Buy_Signal'] = np.nan
    df['Sell_Signal'] = np.nan

    # Check for divergence over a rolling window
    for i in range(look_back, len(df)):
        # Get the last 'look_back' number of candles for price and RSI
        price_segment = df['close'][i-look_back:i]
        rsi_segment = df['RSI'][i-look_back:i]

        # Detect Bullish Divergence: Lower low in price and higher low in RSI
        if (price_segment.min() < df['close'][i]) and (rsi_segment.min() > df['RSI'][i]):
            df.at[i, 'Buy_Signal'] = df['close'][i]

        # Detect Bearish Divergence: Higher high in price and lower high in RSI
        if (price_segment.max() > df['close'][i]) and (rsi_segment.max() < df['RSI'][i]):
            df.at[i, 'Sell_Signal'] = df['close'][i]

    return df

# Example usage
df = pd.DataFrame({
    'date': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'close': np.random.randn(100).cumsum() + 100  # Random walk to simulate prices
})

df = detect_rsi_divergence(df)
print(df[['date', 'close', 'RSI', 'Buy_Signal', 'Sell_Signal']].tail(20))


         date   close     RSI  Buy_Signal  Sell_Signal
80 2023-03-22 81.5468 67.6869         NaN          NaN
81 2023-03-23 81.6164 63.0044         NaN          NaN
82 2023-03-24 81.1767 60.7122         NaN          NaN
83 2023-03-25 80.4917 58.4916         NaN          NaN
84 2023-03-26 82.0289 60.7365         NaN          NaN
85 2023-03-27 79.8235 49.3062         NaN          NaN
86 2023-03-28 80.0984 46.7782     80.0984          NaN
87 2023-03-29 81.8699 55.8695         NaN          NaN
88 2023-03-30 80.9972 52.9118         NaN          NaN
89 2023-03-31 81.9964 48.4395         NaN          NaN
90 2023-04-01 82.3072 53.7919         NaN          NaN
91 2023-04-02 83.4874 56.0980         NaN          NaN
92 2023-04-03 80.7307 47.6185         NaN          NaN
93 2023-04-04 79.4221 40.9909         NaN          NaN
94 2023-04-05 79.8982 44.4631         NaN          NaN
95 2023-04-06 79.9162 44.2699         NaN          NaN
96 2023-04-07 81.8034 51.9242         NaN          NaN
97 2023-04

In [79]:
df = bnf_1min
df['datetime'] = df['datetime'].dt.tz_localize(None)
df = resample(pl.DataFrame(df), '3m')
df = df.to_pandas()
df = calculate_signals(df)

In [80]:
df.tail()

,datetime,open,high,low,close,RSI,Buy_Signal,Sell_Signal
242516,2024-12-31 15:15:00,50875.9500,50891.2000,50862.6500,50869.6500,59.6903,NaN,50869.6500
242517,2024-12-31 15:18:00,50869.5000,50898.0500,50852.7000,50896.0500,64.4862,NaN,NaN
242518,2024-12-31 15:21:00,50895.8500,50899.4500,50859.3500,50862.6000,55.8573,NaN,NaN
242519,2024-12-31 15:24:00,50865.6000,50885.8500,50853.8500,50875.5000,51.5126,NaN,NaN
242520,2024-12-31 15:27:00,50875.2500,50906.3500,50857.2000,50884.3000,56.4895,NaN,NaN


In [81]:
df_buy = df[df['Buy_Signal'] > 0]
df_buy.tail()

,datetime,open,high,low,close,RSI,Buy_Signal,Sell_Signal
242448,2024-12-31 11:51:00,50839.9000,50848.6500,50809.6500,50817.4000,40.0683,50817.4000,NaN
242449,2024-12-31 11:54:00,50815.9500,50815.9500,50782.9000,50794.3000,29.3186,50794.3000,NaN
242475,2024-12-31 13:12:00,50839.4500,50846.2000,50826.4500,50840.6000,43.4701,50840.6000,NaN
242492,2024-12-31 14:03:00,50877.3500,50905.4500,50873.4500,50897.7000,56.4737,50897.7000,NaN
242494,2024-12-31 14:09:00,50904.7000,50911.3000,50883.5000,50892.6000,51.6015,50892.6000,NaN
